In [8]:
import numpy as np
from src.datasets.tensor_storage import TensorStorage
from tqdm import tqdm
from sklearn.decomposition import IncrementalPCA
import torch
import pandas as pd
import os

In [2]:
np.random.seed(42)
torch.manual_seed(42)

In [5]:
store = TensorStorage("storages/molformer_dgsm")

In [6]:
store

TensorStorage at 'storages/molformer_dgsm'
Description: 
Number of tensors: 1576904
Chunk size: 6.00 MB
Total storage size: 4619.84 MB
Tensor shape: [768]

In [7]:
store[3]

array([ 1.39658916e+00, -3.77277374e-01, -5.93409121e-01,  4.68972832e-01,
       -9.36067760e-01, -2.75446653e-01, -2.84941941e-01, -4.63555343e-02,
       -8.73108387e-01,  8.54577124e-02, -8.31386268e-01,  3.72124314e-01,
        8.14772427e-01,  3.72868814e-02, -5.65502822e-01,  8.63719940e-01,
        3.12404424e-01,  4.66239840e-01, -9.22818601e-01, -5.93148053e-01,
       -2.77471453e-01,  3.38346481e-01, -3.94446915e-03, -5.60185552e-01,
       -5.35604537e-01,  6.90113753e-04, -2.70525068e-01,  9.71472442e-01,
       -4.02895242e-01,  6.56837761e-01, -6.71515942e-01, -6.93513393e-01,
        4.01010394e-01,  5.93517005e-01,  5.70327580e-01,  4.18171406e-01,
        3.10023218e-01,  2.53105704e-02,  3.60354066e-01, -9.23711061e-01,
        8.81871521e-01, -8.07749555e-02, -1.64673254e-01, -6.06982529e-01,
       -1.16132295e+00,  1.91671896e+00, -1.37960386e+00,  1.41151398e-01,
       -2.81311333e-01,  1.24079585e-01,  4.10990834e-01,  5.19646704e-01,
        1.74448639e-02, -

In [15]:
store[3].shape

(768,)

In [9]:
metadata_df = store.load_metadata_table()

In [10]:
metadata_df.head() # success - was there error during embedding creation

,smiles,success,index,batch_id,processing_time,tensor_idx
0,CCO,True,0,0,1.734394e+09,0
1,C,True,1,0,1.734394e+09,1
2,CO,True,2,0,1.734394e+09,2
3,NCCS,True,3,0,1.734394e+09,3
4,NCCN,True,4,0,1.734394e+09,4


In [11]:
non_error_index = metadata_df[metadata_df['success']==True].index

In [12]:
len(non_error_index), metadata_df.shape[0]

(1576904, 1576904)

In [13]:
def load_embeddings_batch(store, non_error_index, batch_size=1000):
    """
    Load embeddings in batches and yield them
    """
    for i in range(0, len(non_error_index), batch_size):
        batch_indices = non_error_index[i:i + batch_size]
        batch_embeddings = []
        
        for idx in batch_indices:
            emb = store[idx]
            if isinstance(emb, torch.Tensor):
                emb = emb.numpy()
            batch_embeddings.append(emb.reshape(-1))
            
        yield np.array(batch_embeddings)

In [17]:
def compute_principal_components(store, non_error_index, batch_size=1000):
    """
    Compute all 300 principal components of the embedding space using IncrementalPCA
    """
    # Initialize IncrementalPCA with full dimensionality
    ipca = IncrementalPCA(n_components=768)  # Get all 300 components
    
    # Fit PCA incrementally
    for batch in tqdm(load_embeddings_batch(store, non_error_index, batch_size), 
                     desc="Computing PCA", 
                     total=len(non_error_index)//batch_size + 1):
        ipca.partial_fit(batch)
    
    # Get explained variance ratio and eigenvectors
    explained_variance_ratio = ipca.explained_variance_ratio_
    eigenvectors = ipca.components_  # Shape will be (300, 300)
    
    return eigenvectors, explained_variance_ratio, ipca

In [18]:
def direct_egv_comp(store, non_error_index, batch_size=1000):
    """
    Compute eigenvectors through direct covariance matrix calculation
    """
    # Step 1: Calculate mean vector
    print("Computing mean vector...")
    mean_vector = np.zeros(768)
    total_count = 0
    
    for batch in tqdm(load_embeddings_batch(store, non_error_index, batch_size),
                     desc="Computing mean",
                     total=len(non_error_index)//batch_size + 1):
        mean_vector += np.sum(batch, axis=0)
        total_count += batch.shape[0]
    
    mean_vector /= total_count
    
    # Step 2: Compute covariance matrix block by block
    print("Computing covariance matrix...")
    cov_matrix = np.zeros((768, 768))
    
    for batch in tqdm(load_embeddings_batch(store, non_error_index, batch_size),
                     desc="Computing covariance",
                     total=len(non_error_index)//batch_size + 1):
        # Center the batch
        centered_batch = batch - mean_vector
        # Update covariance matrix
        cov_matrix += centered_batch.T @ centered_batch
    
    cov_matrix /= (total_count - 1)
    
    # Step 3: Compute eigenvectors and eigenvalues
    print("Computing eigenvectors...")
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    
    # Sort by eigenvalues in descending order
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Calculate explained variance ratio
    explained_variance_ratio = eigenvalues / np.sum(eigenvalues)
    
    return eigenvectors.T, explained_variance_ratio, None  # None instead of ipca

In [19]:
def find_nearest_molecules(store, non_error_index, eigenvectors):
    """
    Find the nearest molecules to each eigenvector using KDTree for efficient nearest neighbor search
    """
    from sklearn.neighbors import KDTree
    
    # First, collect all embeddings into a matrix
    print("Building embeddings matrix...")
    all_embeddings = []
    for idx in tqdm(non_error_index, desc="Loading embeddings"):
        emb = store[idx]
        if isinstance(emb, torch.Tensor):
            emb = emb.numpy()
        emb = emb.reshape(-1)
        
        # Normalize embedding
        emb = emb / np.linalg.norm(emb)
        all_embeddings.append(emb)
    
    all_embeddings = np.array(all_embeddings)
    
    # Print diagnostics for embeddings
    print("\nEmbeddings statistics:")
    print(f"Shape: {all_embeddings.shape}")
    print(f"Mean norm: {np.mean([np.linalg.norm(emb) for emb in all_embeddings])}")
    print(f"Mean: {np.mean(all_embeddings)}")
    print(f"Std: {np.std(all_embeddings)}")
    
    # Normalize eigenvectors
    print("\nEigenvectors statistics before normalization:")
    print(f"Shape: {eigenvectors.shape}")
    print(f"Mean norm: {np.mean([np.linalg.norm(vec) for vec in eigenvectors])}")
    print(f"Mean: {np.mean(eigenvectors)}")
    print(f"Std: {np.std(eigenvectors)}")
    
    normalized_eigenvectors = eigenvectors.copy()
    for i in range(len(normalized_eigenvectors)):
        normalized_eigenvectors[i] = normalized_eigenvectors[i] / np.linalg.norm(normalized_eigenvectors[i])
    
    print("\nEigenvectors statistics after normalization:")
    print(f"Mean norm: {np.mean([np.linalg.norm(vec) for vec in normalized_eigenvectors])}")
    
    # Build KDTree
    print("\nBuilding KDTree...")
    tree = KDTree(all_embeddings)
    
    # Query tree for each eigenvector
    print("Finding nearest neighbors...")
    nearest_molecules = []
    nearest_distances = []
    
    for eigenvector in tqdm(normalized_eigenvectors, desc="Querying KDTree"):
        distances, indices = tree.query(eigenvector.reshape(1, -1), k=5)  # Get top 5 for debugging
        
        # Print first few matches for debugging
        if len(nearest_molecules) < 3:
            print(f"\nEigenvector {len(nearest_molecules)} top matches:")
            for d, i in zip(distances[0], indices[0]):
                orig_idx = non_error_index[i]
                print(f"Distance: {d:.4f}, SMILES: {metadata_df.iloc[orig_idx]['smiles']}")
        
        nearest_molecules.append(non_error_index[indices[0][0]])
        nearest_distances.append(distances[0][0])
    
    return nearest_molecules, nearest_distances


In [20]:
non_error_index = metadata_df[metadata_df['success']==True].index.values

In [ ]:
print("Computing principal components...")
eigenvectors, explained_variance_ratio, ipca = compute_principal_components(store, non_error_index)

In [22]:
print("Computing eigenvectors directly...")
eigenvectors, explained_variance_ratio, ipca = direct_egv_comp(store, non_error_index)

Computing eigenvectors directly...
Computing mean vector...


Computing mean: 100%|██████████| 1577/1577 [00:27<00:00, 57.28it/s]


Computing covariance matrix...


Computing covariance: 100%|██████████| 1577/1577 [00:32<00:00, 48.10it/s]


Computing eigenvectors...


In [23]:
print("Finding nearest molecules to eigenvectors...")
nearest_molecules, nearest_distances = find_nearest_molecules(store, non_error_index, eigenvectors)

Finding nearest molecules to eigenvectors...
Building embeddings matrix...


Loading embeddings: 100%|██████████| 1576904/1576904 [00:16<00:00, 93430.50it/s] 



Embeddings statistics:
Shape: (1576904, 768)
Mean norm: 1.0
Mean: -0.0001453390868846327
Std: 0.036080531775951385

Eigenvectors statistics before normalization:
Shape: (768, 768)
Mean norm: 1.0
Mean: 4.615175322810023e-05
Std: 0.03608436231041096

Eigenvectors statistics after normalization:
Mean norm: 1.0

Building KDTree...
Finding nearest neighbors...


Querying KDTree:   0%|          | 1/768 [00:01<22:31,  1.76s/it]


Eigenvector 0 top matches:
Distance: 0.9129, SMILES: CC1OC(OC2C(O)C(OC3OC(CO)C(O)C(O)C3O)C(OC3CCC4(C)C(CCC5(C)C4CC=C4C6CC(C)(C)CC(OC(=O)C=C(C)C)C6(CO)C(O)C(O)C54C)C3(C)C)OC2C(O)=O)C(O)C(O)C1O
Distance: 0.9170, SMILES: CC1OC(OC2CCC3(C)C(CCC4(C)C3=CC=C3C5CC(C)(C)CCC5(CO)C(O)CC43C)C2(C)CO)C(O)C(OC2OC(CO)C(OC3OCC(O)C(O)C3O)C(O)C2O)C1O
Distance: 0.9187, SMILES: COC(C)(C)C=CC(O)C(C)(O)C1CCC2(C)C1CCC1C3(C)CCC(OC4OC(CO)C(O)C(O)C4OC4OC(C)C(O)C(O)C4O)C(C)(C)C3CC(O)C21C
Distance: 0.9197, SMILES: CC(C)=CCCC(C)(O)C1CCC2(C)C1C(O)CC1C3(C)CCC(OC4OC(CO)C(O)C(O)C4OC4OC(CO)C(O)C(O)C4OC4OCC(O)C(O)C4O)C(C)(C)C3CCC21C
Distance: 0.9211, SMILES: CC=C(C)C(=O)OC1C(OC(C)=O)C(C)(C)CC2C3=CCC4C5(C)CCC(OC6OC(C(O)C(O)C6OC6OC(C)C(O)C(O)C6O)C(O)=O)C(C)(C)C5CCC4(C)C3(C)C(O)C(O)C12CO


Querying KDTree:   0%|          | 2/768 [00:03<23:34,  1.85s/it]


Eigenvector 1 top matches:
Distance: 0.8524, SMILES: CC1(C)Oc2cc(O)c3C(=O)c4cc(O)ccc4Oc3c2C=C1
Distance: 0.8587, SMILES: COc1c(O)cc(O)c2C(=O)C=C(Oc12)c1ccccc1
Distance: 0.8590, SMILES: CC1(C)Oc2cc(O)c3C(=O)c4ccc(O)cc4Oc3c2C=C1
Distance: 0.8596, SMILES: CC1(C)Oc2cc3C(=O)c4ccccc4Oc3c(O)c2C=C1
Distance: 0.8597, SMILES: CC1(C)Oc2cc(O)c3C(=O)c4ccccc4Oc3c2C=C1


Querying KDTree:   0%|          | 3/768 [00:05<24:25,  1.92s/it]


Eigenvector 2 top matches:
Distance: 1.0235, SMILES: COc1ccc(C=C2SC(=S)N(CC(=O)OCC(=O)Nc3ccc(C)cc3Cl)C2=O)cc1
Distance: 1.0364, SMILES: COc1ccc(C=C2SC(=S)N(CC(=O)OCC(=O)Nc3cc(Cl)cc(Cl)c3)C2=O)cc1
Distance: 1.0589, SMILES: CCc1nnc(NC(=O)CN2C(=S)SC(=Cc3ccc(OC)c(OC)c3)C2=O)s1
Distance: 1.0598, SMILES: CCc1nnc(NC(=O)CN2C(=S)SC(=Cc3cc(OC)c(OC)c(OC)c3)C2=O)s1
Distance: 1.0616, SMILES: CCc1ccc(C=C2SC(=S)N(CC(=O)Nc3nc4ccccc4s3)C2=O)cc1


Querying KDTree: 100%|██████████| 768/768 [25:36<00:00,  2.00s/it]


In [24]:
results_df = pd.DataFrame({
        'eigenvector_idx': range(len(nearest_molecules)),
        'nearest_molecule_idx': nearest_molecules,
        'distance': nearest_distances,
        'explained_variance_ratio': explained_variance_ratio,
        'cumulative_variance_ratio': np.cumsum(explained_variance_ratio),
        'smiles': [metadata_df.iloc[idx]['smiles'] for idx in nearest_molecules]
    })

In [25]:
results_df = results_df.sort_values('explained_variance_ratio', ascending=False)

In [26]:
results_df

,eigenvector_idx,nearest_molecule_idx,distance,explained_variance_ratio,cumulative_variance_ratio,smiles
0,0,1555678,0.912870,7.523119e-02,0.075231,CC1OC(OC2C(O)C(OC3OC(CO)C(O)C(O)C3O)C(OC3CCC4(...
1,1,538160,0.852413,6.704994e-02,0.142281,CC1(C)Oc2cc(O)c3C(=O)c4cc(O)ccc4Oc3c2C=C1
2,2,1196081,1.023466,5.369856e-02,0.195980,COc1ccc(C=C2SC(=S)N(CC(=O)OCC(=O)Nc3ccc(C)cc3C...
3,3,1565124,1.223687,3.813828e-02,0.234118,OC1C2OC(=O)c3c1c(O)c(O)c(O)c3-c1c(O)c(O)c(O)c3...
4,4,641064,1.043033,3.514329e-02,0.269261,O=C1CN(CC(=O)N1)S(=O)(=O)c1ccc(cc1)N(=O)=O
...,...,...,...,...,...,...
763,763,228,1.399401,1.023342e-05,0.999970,CSCCC=O
764,764,102,1.405356,1.006060e-05,0.999980,CCON
765,765,42,1.408313,9.972632e-06,0.999990,COCOC
766,766,1249,1.400865,9.745502e-06,1.000000,[O-][N+]#N


In [27]:
result_dir = "results/molformer_dgsm"
os.makedirs(result_dir, exist_ok=True)

In [28]:
results_df.to_csv(os.path.join(result_dir, 'molformer_pca_results.csv'), index=False)

In [29]:
np.save(os.path.join(result_dir, 'molformer_eigenvectors.npy'), eigenvectors)